## 1. Column Mapping

#### Setup & Load Data

In [1]:
import pandas as pd 
import numpy as np 
from pathlib import Path 
import re

In [2]:
ROOT = Path(r"D:\DATA SCIENCE PROJECTS\FHI_SOUTH_AFRICA")
DATA_RAW = ROOT / "data" / "raw"
OUT_PROF = ROOT / "outputs" / "profiling"
OUT_PROF.mkdir(parents=True, exist_ok=True)

In [3]:
train_path = DATA_RAW / "Train.csv"
test_path = DATA_RAW / "Test.csv"

In [4]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [5]:
print("Train:", train_df.shape, "| Test:", test_df.shape)
train_df.head(2)

Train: (9618, 39) | Test: (2405, 38)


,ID,country,owner_age,attitude_stable_business_environment,attitude_worried_shutdown,compliance_income_tax,perception_insurance_doesnt_cover_losses,perception_cannot_afford_insurance,personal_income,business_expenses,...,has_internet_banking,has_debit_card,future_risk_theft_stock,business_age_months,medical_insurance,funeral_insurance,motivation_make_more_money,uses_friends_family_savings,uses_informal_lender,Target
0,ID_3CFL0U,eswatini,63.0,Yes,No,No,No,Yes,3000.0,6000.0,...,Never had,Never had,NaN,6.0,Never had,Used to have but don’t have now,NaN,Never had,Never had,Low
1,ID_XWI7G3,zimbabwe,39.0,No,Yes,Yes,No,Yes,NaN,NaN,...,NaN,NaN,No,3.0,Never had,Never had,NaN,NaN,NaN,Medium


#### Utility `->` Build Expected Feature List From Train

In [6]:
TARGET_COL = "Target"
COUNTRY_COL = "Country"

# Canonical feature list = train columns excluding target
CANONICAL_FEATURES = [c for c in train_df.columns if c != TARGET_COL]

print("Canonical Feature Count:", len(CANONICAL_FEATURES))
print("First 10 Canonical Features:", CANONICAL_FEATURES[:10])

Canonical Feature Count: 38
First 10 Canonical Features: ['ID', 'country', 'owner_age', 'attitude_stable_business_environment', 'attitude_worried_shutdown', 'compliance_income_tax', 'perception_insurance_doesnt_cover_losses', 'perception_cannot_afford_insurance', 'personal_income', 'business_expenses']


#### Schema Mapping (ID `->` business_id)

In [7]:
def map_schema(df: pd.DataFrame) -> tuple[pd.DataFrame, list[str]]:
    """
    Standardizes identifier column naming:
    - If ID exixts, rename to business_id
    -If busisness_id exists, keep
    -If neither exists, generate business_id
    """
    notes = []

    df = df.copy()

    if "business_id" in df.columns:
        notes.append("business_id already present.")
    elif "ID" in df.columns:
        df =df.rename(columns={"ID": "business_id"})
        notes.append("Renamed ID -> business_id.")
    else:
        # generate stable ids based on row index
        df["business_id"] = [f"auti_{i:06d}" for i in range(1, len(df) + 1)]
        notes.append("No ID/business_id found. Generated business_id as auto_000001...")

    return df, notes

## 2. Normalize Text Values

In [8]:
MISSING_TOKENS = {
    "", " ", "  ", "n/a", "na", "none", "null", "nan", "refused", "prefer not to say", "not applicable"
    }
def normalize_text_value(x):
    if pd.isna(x):
        return "missing"
    
    # cast to str
    s = str(x).strip()

    # Normalize curly apostrophes to straight
    s = s.replace("’", "'").replace("‘", "'")

    # collapse repeated whitespace
    s = re.sub(r"\s+", " ", s).strip()

    # Handle missing tokens
    if s.lower() in MISSING_TOKENS:
        return "missing"
    
    # Standardize Don't/Don`t to "dont_know" if it's a "don't know" style token
    if s.lower() in ["don't know", "dont know", "don’t knoww"]:
        return "dont_know"
    return s


In [9]:
def normalize_categoricals(df: pd.DataFrame, categorical_cols: list[str]) -> tuple[pd.DataFrame, list[str]]:
    """
    Normalize categorical columns to reduce duplicate categories and stabilize encoding
    Returns cleaned df + notes
    """

    df = df.copy()
    notes = []
    for c in categorical_cols:
        # Only normalize non-numeric columns

        df[c] = df[c].astype("object").map(normalize_text_value)
    notes.append(f"Normalized {len(categorical_cols)} categorical columns (apostrophes, midding tokens, dont_know).")
    return df, notes 

#### Identify Numeric vs Categorical Columns From Profiling Logic

In [10]:
# Use train to infer types
feature_cols = [c for c in train_df.columns if c != TARGET_COL]

numeric_cols = train_df[feature_cols].select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = [c for c in feature_cols if c not in numeric_cols]

print("Numeric:", numeric_cols)
print("Categorical count:", len(categorical_cols))

Numeric: ['owner_age', 'personal_income', 'business_expenses', 'business_turnover', 'business_age_years', 'business_age_months']
Categorical count: 32


## Safe Feature derivation (Business Age Conversion)

In [11]:
def derive_features(df: pd.DataFrame) -> tuple[pd.DataFrame, list[str]]:
    """
    Derives business_age_years/months where possible.
    Does NOT guess money units or invent financial values.
    """
    df = df.copy()
    notes = []

    if "business_age_months" in df.columns and "business_age_years" in df.columns:
        # if months missing but years present
        months_missing = df["business_age_months"].isna() | (df["business_age_months"] == "missing")
        years_numeric = pd.to_numeric(df["business_age_years"], errors = "coerce")

    # If years missing but months present
    years_missing = df["business_age_years"].isna() | (df["business_age_years"] == "missing")
    months_numeric = pd.to_numeric(df["business_age_months"], errors="coerce")

    if years_missing.any() and months_numeric.notna().any():
        df.loc[years_missing, "business_age_years"] = np.floor(months_numeric[years_missing] / 12)
        notes.append("Derived business_age_years from business_age_months where missing.")

    return df, notes

## Compatibility Grading (`1 / 2 / 3`)

In [12]:
ACCESS_RESILIENCE_SIGNALS = [
    "has_mobile_money",
    "has_internet_banking",
    "has_debit_card",
    "has_loan_account",
    "funera;_insurance",
    "medical_insurance",
    "motor_vehicle_insurance",
    "uses_informal_lender",
    "uses_friends_family_savings",
    "current_problem_cash_flow",
    "problem_sourcing_money",
]

BASIC_SIGNALS = ["owner_age", "business_age_years", "business_age_months"]
FINANCIAL_ACTIVITY = ["business_turnover", "business_expenses", "personal_income"]

def count_present_signals(df: pd.DataFrame, cols: list[str]) -> int:
    present = 0
    for c in cols:
        if c in df.columns:
            # present if column exists and not fully missing
            s = df[c]
            if s.isna().mean() < 0.95: # not missing entirely
                present += 1
    return present

In [13]:
def grade_compatibility(df: pd.DataFrame) -> tuple[int, list[str]]:
    """
    Returns (grade, notes)
    Grade 1: enough signals
    Grade 2: missing some but derivable/defaultable
    Grade 3: not compatible (cannot score)
    """

    notes = []

    df = df.copy()

    # Hard fail: country missing
    if COUNTRY_COL not in df.columns:
        return 3, ["Missing required column: country. Cannot score."]
    
    # check core groups: basics, financial activity, access/resilience signals
    basic_present = count_present_signals(df, BASIC_SIGNALS)
    fin_present = count_present_signals(df, FINANCIAL_ACTIVITY)
    access_present = count_present_signals(df, ACCESS_RESILIENCE_SIGNALS)

    notes.append(f"Signal Summary:basics={basic_present}, financial_activity={fin_present}, access_resilience={access_present}.")

    # Grade 1 rule (reliable scoring)
    if basic_present >=1 and fin_present >= 1 and access_present >= 2:
        notes.append("Grade 1: Sufficient signals for reliable scoring.")
        return 1, notes
    
    # Grade 2 rule (partial but stable)
    # Allow partial if at least: basics>=1 AND (fin_present>=1 OR access_present>=2)
    # because some datasets may have strong access signals but no money values, or vice versa.
    if basic_present >= 1 and (fin_present >= 1 or access_present >= 2):
        notes.append("Grade 2: Partial signals but derivable/defaultable.")
        return 2, notes

    # Grade 3 rule (incompatible)
    notes.append("Grade 3: Insufficient signals; cannot score reliably.")
    return 3, notes

## Building a Single `Contract Engine` Function

In [14]:
def run_contract_engine(df: pd.DataFrame, categorical_cols: list[str]) -> dict:
    """
    Runs the shared trunk:
    1) Schema mapping
    2) Normalization
    3) Derivation
    4) Compatibility grading
    Returns dict with cleaned df + grade + notes
    """

    notes_all = []

    df1, notes = map_schema(df)
    notes_all += notes

    # ensure all canonical columns exist (create missing columns as NA) to stabilize downstream
    for col in CANONICAL_FEATURES:
        if col not in df1.columns:
            df1[col] = np.nan
            notes_all.append(f"Added missing canonical column: '{col}' as NaN (for schema alignment).")

    # Normalize categoricals
    df2, notes = normalize_categoricals(df1, categorical_cols)
    notes_all += notes

    # Feature derivation
    df3, notes = derive_features(df2)
    notes_all += notes

    # Compatibility grading
    grade, grade_notes = grade_compatibility(df3)
    notes_all += grade_notes

    return {
        "df": df3,
        "compatibility_grade": grade,
        "compatibility_notes": notes_all
    }

### Test Engine on current data `Train.csv`/`Test.csv`

In [15]:
train_out = run_contract_engine(train_df.drop(columns=[TARGET_COL]), categorical_cols)
test_out = run_contract_engine(test_df, categorical_cols)

print("Train Compatibility Grade:", train_out["compatibility_grade"])
print("Test Compatibility Grade:", test_out["compatibility_grade"])

print("\nTrain notes (first 10):")
for n in train_out["compatibility_notes"][:10]:
    print("-", n)

# Save notes for audit trail
pd.Series(train_out["compatibility_notes"]).to_csv(OUT_PROF / "contract_engine_notes_train.csv", index=False, header="note")
pd.Series(test_out["compatibility_notes"]).to_csv(OUT_PROF / "contract_engine_notes_test.csv", index=False, header="note")

Train Compatibility Grade: 3
Test Compatibility Grade: 3

Train notes (first 10):
- Renamed ID -> business_id.
- Added missing canonical column: 'ID' as NaN (for schema alignment).
- Normalized 32 categorical columns (apostrophes, midding tokens, dont_know).
- Derived business_age_years from business_age_months where missing.
- Missing required column: country. Cannot score.


### Force a Grade 3 example (drop country)

In [17]:
broken = test_df.copy()

country_col = next((c for c in broken.columns if c.strip().lower() == "country"), None)
print("Detected country column:", country_col)

# drop country column if found
if country_col is not None:
    broken = broken.drop(columns=[country_col])
else:
    print("No country column found")

broken_out = run_contract_engine(broken, categorical_cols)
print("Broken grade:", broken_out["compatibility_grade"])
print("Key notes:")
for n in broken_out["compatibility_notes"][-5:]:
    print("-", n)


Detected country column: country
Broken grade: 3
Key notes:
- Added missing canonical column: 'ID' as NaN (for schema alignment).
- Added missing canonical column: 'country' as NaN (for schema alignment).
- Normalized 32 categorical columns (apostrophes, midding tokens, dont_know).
- Derived business_age_years from business_age_months where missing.
- Missing required column: country. Cannot score.
